In [1]:
import pandas as pd
import numpy as np
import torch
import datetime
import itertools
import os
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [2]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

print(f"Training on {DEVICE} using PyTorch {torch.__version__}")
print('main2_mlp_federated')

Training on cpu using PyTorch 1.13.1+cpu
main2_mlp_federated


In [3]:
# Load data
X = torch.load('./dataset2/X_mlp.pt')
y = torch.load('./dataset2/y_mlp.pt')

In [4]:
def test(net, x_test, y_test):
  with torch.no_grad():
    y_pred = net(x_test)
    y_pred = y_pred.detach().numpy()
    y_test = y_test.detach().numpy()

    mse = mean_squared_error(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    return mse, auc

In [5]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=4)
#print(type(X_train))

In [6]:
# Define MLP FL simulation wrapper
from fl_simu import mlp_simulation2

def run_simulation(split, X_train, y_train, X_test, y_test,dir='default', num_epoch=1, batch_size=32, num_rounds=1, dropout=False, history_dir='./history/example.txt'):
    return mlp_simulation2(split, X_train, y_train, X_test, y_test, dir, num_epoch=num_epoch, batch_size=batch_size, num_rounds=num_rounds, dropout=dropout, history_dir=history_dir)

D:\FD\venv\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [7]:
num_epochs_list = [2]
batch_size_list = [32]
num_rounds_list = [25] # 15
dropout_list = [0]

In [8]:

test_name = 'dataset 2 MLP federated'

directory_path = 'model_checkpoints2/' + test_name

# # Check if the directory exists, and if not, create it
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

def do_fl(splitting, X=X, y=y):

    start = datetime.datetime.now()
    mses = []
    aucs = []

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    X = X.to_numpy()
    y = y.to_numpy()

    flag = 1

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X, y)):
        print(f"Fold {fold_idx + 1}:")

        if flag == 2:
            break

        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        X_train = torch.tensor(X_train)
        X_test = torch.tensor(X_test)
        y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
        y_test = torch.reshape(torch.tensor(y_test), (-1, 1))

        X_test = X_test.to(DEVICE)
        y_test = y_test.to(DEVICE)

        print(f'x_test: {X_test.device}')
        print(f'y_test: {y_test.device}')

        for batch_size, num_epoch, num_rounds, dropout in itertools.product(
                batch_size_list,
                num_epochs_list,
                num_rounds_list,
                dropout_list):


            filename = 'mlp_batch_'+ str(batch_size) + '_epochs_' + str(num_epoch) + '_rounds_' + str(num_rounds) + '_loss.csv'

            history_dir = './history/dataset2/' + filename


            model_fl = run_simulation(splitting, X_train, y_train, X_test, y_test,test_name, batch_size=batch_size, num_epoch=num_epoch, num_rounds=num_rounds, history_dir=history_dir, dropout=dropout, )

            model_fl.cpu()

            X_test = X_test.cpu()
            y_test = y_test.cpu()

            mse, auc= test(model_fl, X_test, y_test)
            mses.append(mse)
            aucs.append(auc)

            flag += 1

    end = datetime.datetime.now()
    time = end - start

    print('splitting', splitting)
    print('time', time)
    print('mse', mses)
    print('auc', aucs)



In [9]:
splitting = [0.5, 0.5]
do_fl(splitting)

Fold 1:
x_test: cpu
y_test: cpu


INFO flwr 2023-08-31 17:47:30,148 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)


Client 0 trainset shape torch.Size([226178, 120])
Client 1 trainset shape torch.Size([226118, 120])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 17:47:38,221	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
INFO flwr 2023-08-31 17:47:48,710 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1233194190.0, 'object_store_memory': 616597094.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-08-31 17:47:48,713 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 17:47:48,714 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 17:47:57,083 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 17:47:57,087 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 17:47:57,090 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 17:47:57,093 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 17:49:04,657 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 17:49:04,667 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  2
server round is  24
loss is  0.19561509788036346


DEBUG flwr 2023-08-31 18:14:01,972 | server.py:236 | fit_round 25 received 2 results and 0 failures
DEBUG flwr 2023-08-31 18:14:01,989 | server.py:173 | evaluate_round 25: strategy sampled 2 clients (out of 2)


agrregate_fit results list length is  2
Saving round 25 aggregated_parameters...


DEBUG flwr 2023-08-31 18:14:04,081 | server.py:187 | evaluate_round 25 received 2 results and 0 failures
INFO flwr 2023-08-31 18:14:04,083 | server.py:153 | FL finished in 1566.9904870000028
INFO flwr 2023-08-31 18:14:04,085 | app.py:225 | app_fit: losses_distributed [(1, 0.21264789998531342), (2, 0.20469626784324646), (3, 0.20167231559753418), (4, 0.19934159517288208), (5, 0.20020543038845062), (6, 0.1985892653465271), (7, 0.19811365008354187), (8, 0.19738562405109406), (9, 0.1976688951253891), (10, 0.19732984900474548), (11, 0.19633086025714874), (12, 0.19616657495498657), (13, 0.1968826949596405), (14, 0.19637349247932434), (15, 0.19760634005069733), (16, 0.19632773101329803), (17, 0.19700972735881805), (18, 0.19847777485847473), (19, 0.1949663907289505), (20, 0.1956903338432312), (21, 0.1952711045742035), (22, 0.19428500533103943), (23, 0.1960931122303009), (24, 0.19561509788036346), (25, 0.19590094685554504)]
INFO flwr 2023-08-31 18:14:04,086 | app.py:226 | app_fit: metrics_distri

agrregate_evaluate results list length is  2
server round is  25
loss is  0.19590094685554504
History (loss, distributed):
	round 1: 0.21264789998531342
	round 2: 0.20469626784324646
	round 3: 0.20167231559753418
	round 4: 0.19934159517288208
	round 5: 0.20020543038845062
	round 6: 0.1985892653465271
	round 7: 0.19811365008354187
	round 8: 0.19738562405109406
	round 9: 0.1976688951253891
	round 10: 0.19732984900474548
	round 11: 0.19633086025714874
	round 12: 0.19616657495498657
	round 13: 0.1968826949596405
	round 14: 0.19637349247932434
	round 15: 0.19760634005069733
	round 16: 0.19632773101329803
	round 17: 0.19700972735881805
	round 18: 0.19847777485847473
	round 19: 0.1949663907289505
	round 20: 0.1956903338432312
	round 21: 0.1952711045742035
	round 22: 0.19428500533103943
	round 23: 0.1960931122303009
	round 24: 0.19561509788036346
	round 25: 0.19590094685554504
History (metrics, distributed, evaluate):
{'mse': [(1, 0.42529579997062683), (2, 0.4093925356864929), (3, 0.4033446311

In [10]:
splitting = [0.6, 0.4]
do_fl(splitting)

Fold 1:
x_test: cpu
y_test: cpu


INFO flwr 2023-08-31 18:25:08,527 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)


Client 0 trainset shape torch.Size([271374, 120])
Client 1 trainset shape torch.Size([180922, 120])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 18:25:18,957	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
INFO flwr 2023-08-31 18:25:24,343 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1467428046.0, 'object_store_memory': 733714022.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-08-31 18:25:24,344 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 18:25:24,345 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 18:25:29,657 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 18:25:29,658 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 18:25:29,659 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 18:25:29,660 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 18:26:26,258 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 18:26:26,289 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  2
server round is  24
loss is  0.19393500685691833


DEBUG flwr 2023-08-31 18:47:42,606 | server.py:236 | fit_round 25 received 2 results and 0 failures
DEBUG flwr 2023-08-31 18:47:42,632 | server.py:173 | evaluate_round 25: strategy sampled 2 clients (out of 2)


agrregate_fit results list length is  2
Saving round 25 aggregated_parameters...


DEBUG flwr 2023-08-31 18:47:44,607 | server.py:187 | evaluate_round 25 received 2 results and 0 failures
INFO flwr 2023-08-31 18:47:44,610 | server.py:153 | FL finished in 1334.9507319999975
INFO flwr 2023-08-31 18:47:44,612 | app.py:225 | app_fit: losses_distributed [(1, 0.21843409538269043), (2, 0.20503613352775574), (3, 0.20014217495918274), (4, 0.20163114368915558), (5, 0.19951100647449493), (6, 0.19998368620872498), (7, 0.1987105906009674), (8, 0.19962665438652039), (9, 0.19818896055221558), (10, 0.19789375364780426), (11, 0.19671043753623962), (12, 0.19485190510749817), (13, 0.1938886195421219), (14, 0.1964036077260971), (15, 0.19531501829624176), (16, 0.19647499918937683), (17, 0.1960705667734146), (18, 0.1948298066854477), (19, 0.19435693323612213), (20, 0.19351127743721008), (21, 0.19499942660331726), (22, 0.1946839690208435), (23, 0.19535532593727112), (24, 0.19393500685691833), (25, 0.19464674592018127)]
INFO flwr 2023-08-31 18:47:44,613 | app.py:226 | app_fit: metrics_distr

agrregate_evaluate results list length is  2
server round is  25
loss is  0.19464674592018127
History (loss, distributed):
	round 1: 0.21843409538269043
	round 2: 0.20503613352775574
	round 3: 0.20014217495918274
	round 4: 0.20163114368915558
	round 5: 0.19951100647449493
	round 6: 0.19998368620872498
	round 7: 0.1987105906009674
	round 8: 0.19962665438652039
	round 9: 0.19818896055221558
	round 10: 0.19789375364780426
	round 11: 0.19671043753623962
	round 12: 0.19485190510749817
	round 13: 0.1938886195421219
	round 14: 0.1964036077260971
	round 15: 0.19531501829624176
	round 16: 0.19647499918937683
	round 17: 0.1960705667734146
	round 18: 0.1948298066854477
	round 19: 0.19435693323612213
	round 20: 0.19351127743721008
	round 21: 0.19499942660331726
	round 22: 0.1946839690208435
	round 23: 0.19535532593727112
	round 24: 0.19393500685691833
	round 25: 0.19464674592018127
History (metrics, distributed, evaluate):
{'mse': [(1, 0.43686819076538086), (2, 0.4100722670555115), (3, 0.400284349

In [11]:
splitting = [0.8, 0.2]
do_fl(splitting)

Fold 1:
x_test: cpu
y_test: cpu


INFO flwr 2023-08-31 18:59:37,841 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)


Client 0 trainset shape torch.Size([361836, 120])
Client 1 trainset shape torch.Size([90460, 120])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 18:59:50,687	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
INFO flwr 2023-08-31 18:59:59,332 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1406580327.0, 'object_store_memory': 703290163.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-08-31 18:59:59,335 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 18:59:59,336 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 19:00:06,396 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 19:00:06,397 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 19:00:06,398 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 19:00:06,399 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 19:01:30,653 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 19:01:30,666 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  2
server round is  24
loss is  0.19415810704231262


DEBUG flwr 2023-08-31 19:32:45,153 | server.py:236 | fit_round 25 received 2 results and 0 failures
DEBUG flwr 2023-08-31 19:32:45,166 | server.py:173 | evaluate_round 25: strategy sampled 2 clients (out of 2)


agrregate_fit results list length is  2
Saving round 25 aggregated_parameters...


DEBUG flwr 2023-08-31 19:32:47,771 | server.py:187 | evaluate_round 25 received 2 results and 0 failures
INFO flwr 2023-08-31 19:32:47,773 | server.py:153 | FL finished in 1961.3744161000068
INFO flwr 2023-08-31 19:32:47,776 | app.py:225 | app_fit: losses_distributed [(1, 0.2152034342288971), (2, 0.20097215473651886), (3, 0.19922375679016113), (4, 0.20068593323230743), (5, 0.2004062533378601), (6, 0.1971716582775116), (7, 0.19922059774398804), (8, 0.19562487304210663), (9, 0.19445224106311798), (10, 0.19693316519260406), (11, 0.19798694550991058), (12, 0.19433780014514923), (13, 0.19673453271389008), (14, 0.19619691371917725), (15, 0.1942235827445984), (16, 0.1948210597038269), (17, 0.19463738799095154), (18, 0.19601672887802124), (19, 0.1953795701265335), (20, 0.1936863362789154), (21, 0.19397175312042236), (22, 0.1909964680671692), (23, 0.19573906064033508), (24, 0.19415810704231262), (25, 0.19438911974430084)]
INFO flwr 2023-08-31 19:32:47,777 | app.py:226 | app_fit: metrics_distrib

agrregate_evaluate results list length is  2
server round is  25
loss is  0.19438911974430084
History (loss, distributed):
	round 1: 0.2152034342288971
	round 2: 0.20097215473651886
	round 3: 0.19922375679016113
	round 4: 0.20068593323230743
	round 5: 0.2004062533378601
	round 6: 0.1971716582775116
	round 7: 0.19922059774398804
	round 8: 0.19562487304210663
	round 9: 0.19445224106311798
	round 10: 0.19693316519260406
	round 11: 0.19798694550991058
	round 12: 0.19433780014514923
	round 13: 0.19673453271389008
	round 14: 0.19619691371917725
	round 15: 0.1942235827445984
	round 16: 0.1948210597038269
	round 17: 0.19463738799095154
	round 18: 0.19601672887802124
	round 19: 0.1953795701265335
	round 20: 0.1936863362789154
	round 21: 0.19397175312042236
	round 22: 0.1909964680671692
	round 23: 0.19573906064033508
	round 24: 0.19415810704231262
	round 25: 0.19438911974430084
History (metrics, distributed, evaluate):
{'mse': [(1, 0.4304068684577942), (2, 0.4019443094730377), (3, 0.398447513580

In [12]:
splitting = [0.34, 0.33, 0.33]
do_fl(splitting)

Fold 1:
x_test: cpu
y_test: cpu
Client 0 trainset shape torch.Size([154112, 120])
Client 1 trainset shape torch.Size([144606, 120])


INFO flwr 2023-08-31 19:32:49,859 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)


Client 2 trainset shape torch.Size([144586, 120])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 19:32:59,741	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
INFO flwr 2023-08-31 19:33:07,488 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1716569703.0, 'object_store_memory': 858284851.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-08-31 19:33:07,491 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 19:33:07,493 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 19:33:12,775 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 19:33:12,777 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 19:33:12,779 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 19:33:12,779 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 19:33:47,812 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 19:33:47,831 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  2
server round is  24
loss is  0.19686603546142578


DEBUG flwr 2023-08-31 19:46:05,517 | server.py:236 | fit_round 25 received 2 results and 0 failures
DEBUG flwr 2023-08-31 19:46:05,534 | server.py:173 | evaluate_round 25: strategy sampled 2 clients (out of 3)


agrregate_fit results list length is  2
Saving round 25 aggregated_parameters...


DEBUG flwr 2023-08-31 19:46:07,847 | server.py:187 | evaluate_round 25 received 2 results and 0 failures
INFO flwr 2023-08-31 19:46:07,848 | server.py:153 | FL finished in 775.0683407000033
INFO flwr 2023-08-31 19:46:07,848 | app.py:225 | app_fit: losses_distributed [(1, 0.22898419201374054), (2, 0.2063377946615219), (3, 0.20339186489582062), (4, 0.20107656717300415), (5, 0.1994834989309311), (6, 0.20168977975845337), (7, 0.2000056952238083), (8, 0.1991523951292038), (9, 0.19908280670642853), (10, 0.1975030153989792), (11, 0.19822725653648376), (12, 0.19798968732357025), (13, 0.1995028704404831), (14, 0.19613322615623474), (15, 0.1980186551809311), (16, 0.19841057062149048), (17, 0.1964835226535797), (18, 0.19770242273807526), (19, 0.19467474520206451), (20, 0.19641055166721344), (21, 0.196528360247612), (22, 0.19599954783916473), (23, 0.19872373342514038), (24, 0.19686603546142578), (25, 0.19683164358139038)]
INFO flwr 2023-08-31 19:46:07,849 | app.py:226 | app_fit: metrics_distribute

agrregate_evaluate results list length is  2
server round is  25
loss is  0.19683164358139038
History (loss, distributed):
	round 1: 0.22898419201374054
	round 2: 0.2063377946615219
	round 3: 0.20339186489582062
	round 4: 0.20107656717300415
	round 5: 0.1994834989309311
	round 6: 0.20168977975845337
	round 7: 0.2000056952238083
	round 8: 0.1991523951292038
	round 9: 0.19908280670642853
	round 10: 0.1975030153989792
	round 11: 0.19822725653648376
	round 12: 0.19798968732357025
	round 13: 0.1995028704404831
	round 14: 0.19613322615623474
	round 15: 0.1980186551809311
	round 16: 0.19841057062149048
	round 17: 0.1964835226535797
	round 18: 0.19770242273807526
	round 19: 0.19467474520206451
	round 20: 0.19641055166721344
	round 21: 0.196528360247612
	round 22: 0.19599954783916473
	round 23: 0.19872373342514038
	round 24: 0.19686603546142578
	round 25: 0.19683164358139038
History (metrics, distributed, evaluate):
{'mse': [(1, 0.4579683840274811), (2, 0.4126755893230438), (3, 0.40678372979164

In [13]:
splitting = [0.6, 0.2, 0.2]
do_fl(splitting)

Fold 1:
x_test: cpu
y_test: cpu


INFO flwr 2023-08-31 19:49:31,398 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)


Client 0 trainset shape torch.Size([271428, 120])
Client 1 trainset shape torch.Size([90346, 120])
Client 2 trainset shape torch.Size([90522, 120])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 19:49:42,676	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
INFO flwr 2023-08-31 19:49:48,587 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1950547968.0, 'object_store_memory': 975273984.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-08-31 19:49:48,589 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 19:49:48,590 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 19:49:54,691 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 19:49:54,692 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 19:49:54,694 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 19:49:54,695 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 19:50:22,936 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 19:50:22,944 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  2
server round is  24
loss is  0.1940138190984726


DEBUG flwr 2023-08-31 20:05:16,689 | server.py:236 | fit_round 25 received 2 results and 0 failures
DEBUG flwr 2023-08-31 20:05:16,699 | server.py:173 | evaluate_round 25: strategy sampled 2 clients (out of 3)


agrregate_fit results list length is  2
Saving round 25 aggregated_parameters...


DEBUG flwr 2023-08-31 20:05:18,597 | server.py:187 | evaluate_round 25 received 2 results and 0 failures
INFO flwr 2023-08-31 20:05:18,599 | server.py:153 | FL finished in 923.9044963000051
INFO flwr 2023-08-31 20:05:18,600 | app.py:225 | app_fit: losses_distributed [(1, 0.24486897885799408), (2, 0.20963382720947266), (3, 0.20363111793994904), (4, 0.20036455988883972), (5, 0.20021001994609833), (6, 0.19975994527339935), (7, 0.1997867375612259), (8, 0.19716010987758636), (9, 0.19782836735248566), (10, 0.19857272505760193), (11, 0.19970281422138214), (12, 0.19690851867198944), (13, 0.1961868703365326), (14, 0.19796521961688995), (15, 0.19679273664951324), (16, 0.1972835212945938), (17, 0.1969752162694931), (18, 0.19905903935432434), (19, 0.1948961615562439), (20, 0.19876042008399963), (21, 0.19465726613998413), (22, 0.19502077996730804), (23, 0.1949978768825531), (24, 0.1940138190984726), (25, 0.20003025233745575)]
INFO flwr 2023-08-31 20:05:18,601 | app.py:226 | app_fit: metrics_distrib

agrregate_evaluate results list length is  2
server round is  25
loss is  0.20003025233745575
History (loss, distributed):
	round 1: 0.24486897885799408
	round 2: 0.20963382720947266
	round 3: 0.20363111793994904
	round 4: 0.20036455988883972
	round 5: 0.20021001994609833
	round 6: 0.19975994527339935
	round 7: 0.1997867375612259
	round 8: 0.19716010987758636
	round 9: 0.19782836735248566
	round 10: 0.19857272505760193
	round 11: 0.19970281422138214
	round 12: 0.19690851867198944
	round 13: 0.1961868703365326
	round 14: 0.19796521961688995
	round 15: 0.19679273664951324
	round 16: 0.1972835212945938
	round 17: 0.1969752162694931
	round 18: 0.19905903935432434
	round 19: 0.1948961615562439
	round 20: 0.19876042008399963
	round 21: 0.19465726613998413
	round 22: 0.19502077996730804
	round 23: 0.1949978768825531
	round 24: 0.1940138190984726
	round 25: 0.20003025233745575
History (metrics, distributed, evaluate):
{'mse': [(1, 0.48973795771598816), (2, 0.4192676544189453), (3, 0.4072622358

In [14]:
splitting = [0.8, 0.1, 0.1]
do_fl(splitting)

Fold 1:
x_test: cpu
y_test: cpu


INFO flwr 2023-08-31 20:05:19,897 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)


Client 0 trainset shape torch.Size([361942, 120])
Client 1 trainset shape torch.Size([45238, 120])
Client 2 trainset shape torch.Size([45116, 120])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 20:05:29,574	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
INFO flwr 2023-08-31 20:05:35,671 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 2565075764.0, 'object_store_memory': 1282537881.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-08-31 20:05:35,673 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 20:05:35,675 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 20:05:39,623 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 20:05:39,624 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 20:05:39,625 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 20:05:39,626 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 20:05:50,926 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 20:05:50,934 | fedavg

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  2
server round is  24
loss is  0.19586987793445587


DEBUG flwr 2023-08-31 20:20:46,719 | server.py:236 | fit_round 25 received 2 results and 0 failures
DEBUG flwr 2023-08-31 20:20:46,732 | server.py:173 | evaluate_round 25: strategy sampled 2 clients (out of 3)


agrregate_fit results list length is  2
Saving round 25 aggregated_parameters...


DEBUG flwr 2023-08-31 20:20:48,290 | server.py:187 | evaluate_round 25 received 2 results and 0 failures
INFO flwr 2023-08-31 20:20:48,291 | server.py:153 | FL finished in 908.6644930999901
INFO flwr 2023-08-31 20:20:48,291 | app.py:225 | app_fit: losses_distributed [(1, 0.26130449771881104), (2, 0.2115923911333084), (3, 0.20602862536907196), (4, 0.20028504729270935), (5, 0.20060117542743683), (6, 0.20168235898017883), (7, 0.1991509348154068), (8, 0.19787004590034485), (9, 0.1976289302110672), (10, 0.19670423865318298), (11, 0.19941560924053192), (12, 0.1969105452299118), (13, 0.19624049961566925), (14, 0.1993173509836197), (15, 0.197487935423851), (16, 0.19462528824806213), (17, 0.1979759931564331), (18, 0.1974828839302063), (19, 0.1952129602432251), (20, 0.19280678033828735), (21, 0.19593212008476257), (22, 0.19608920812606812), (23, 0.1926567107439041), (24, 0.19586987793445587), (25, 0.19488553702831268)]
INFO flwr 2023-08-31 20:20:48,292 | app.py:226 | app_fit: metrics_distributed

agrregate_evaluate results list length is  2
server round is  25
loss is  0.19488553702831268
History (loss, distributed):
	round 1: 0.26130449771881104
	round 2: 0.2115923911333084
	round 3: 0.20602862536907196
	round 4: 0.20028504729270935
	round 5: 0.20060117542743683
	round 6: 0.20168235898017883
	round 7: 0.1991509348154068
	round 8: 0.19787004590034485
	round 9: 0.1976289302110672
	round 10: 0.19670423865318298
	round 11: 0.19941560924053192
	round 12: 0.1969105452299118
	round 13: 0.19624049961566925
	round 14: 0.1993173509836197
	round 15: 0.197487935423851
	round 16: 0.19462528824806213
	round 17: 0.1979759931564331
	round 18: 0.1974828839302063
	round 19: 0.1952129602432251
	round 20: 0.19280678033828735
	round 21: 0.19593212008476257
	round 22: 0.19608920812606812
	round 23: 0.1926567107439041
	round 24: 0.19586987793445587
	round 25: 0.19488553702831268
History (metrics, distributed, evaluate):
{'mse': [(1, 0.5226089954376221), (2, 0.4231847822666168), (3, 0.412057250738143

In [15]:
splitting = [0.2, 0.2, 0.2, 0.2, 0.2]
do_fl(splitting)

Fold 1:
x_test: cpu
y_test: cpu
Client 0 trainset shape torch.Size([90398, 120])
Client 1 trainset shape torch.Size([90618, 120])
Client 2 trainset shape torch.Size([90500, 120])


INFO flwr 2023-08-31 20:20:49,366 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)


Client 3 trainset shape torch.Size([90396, 120])
Client 4 trainset shape torch.Size([90384, 120])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 20:20:59,111	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
INFO flwr 2023-08-31 20:21:04,800 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 2250330932.0, 'object_store_memory': 1125165465.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-08-31 20:21:04,802 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 20:21:04,804 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 20:21:08,988 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 20:21:08,989 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 20:21:08,990 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 20:21:08,990 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 20:21:28,207 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 20:21:28,213 | fedavg

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  2
server round is  24
loss is  0.19867227971553802


DEBUG flwr 2023-08-31 20:28:58,980 | server.py:236 | fit_round 25 received 2 results and 0 failures
DEBUG flwr 2023-08-31 20:28:58,992 | server.py:173 | evaluate_round 25: strategy sampled 2 clients (out of 5)


agrregate_fit results list length is  2
Saving round 25 aggregated_parameters...


DEBUG flwr 2023-08-31 20:29:01,153 | server.py:187 | evaluate_round 25 received 2 results and 0 failures
INFO flwr 2023-08-31 20:29:01,155 | server.py:153 | FL finished in 472.16397160000633
INFO flwr 2023-08-31 20:29:01,155 | app.py:225 | app_fit: losses_distributed [(1, 0.2515186071395874), (2, 0.22890549898147583), (3, 0.2104272097349167), (4, 0.2048853486776352), (5, 0.20349368453025818), (6, 0.20247916877269745), (7, 0.19894751906394958), (8, 0.1986970156431198), (9, 0.19881372153759003), (10, 0.19707250595092773), (11, 0.2014366239309311), (12, 0.20076324045658112), (13, 0.20053981244564056), (14, 0.19596511125564575), (15, 0.19598814845085144), (16, 0.19715359807014465), (17, 0.19939061999320984), (18, 0.1958588808774948), (19, 0.2007148712873459), (20, 0.19952644407749176), (21, 0.19650450348854065), (22, 0.19786588847637177), (23, 0.1941422075033188), (24, 0.19867227971553802), (25, 0.19890561699867249)]
INFO flwr 2023-08-31 20:29:01,156 | app.py:226 | app_fit: metrics_distrib

agrregate_evaluate results list length is  2
server round is  25
loss is  0.19890561699867249
History (loss, distributed):
	round 1: 0.2515186071395874
	round 2: 0.22890549898147583
	round 3: 0.2104272097349167
	round 4: 0.2048853486776352
	round 5: 0.20349368453025818
	round 6: 0.20247916877269745
	round 7: 0.19894751906394958
	round 8: 0.1986970156431198
	round 9: 0.19881372153759003
	round 10: 0.19707250595092773
	round 11: 0.2014366239309311
	round 12: 0.20076324045658112
	round 13: 0.20053981244564056
	round 14: 0.19596511125564575
	round 15: 0.19598814845085144
	round 16: 0.19715359807014465
	round 17: 0.19939061999320984
	round 18: 0.1958588808774948
	round 19: 0.2007148712873459
	round 20: 0.19952644407749176
	round 21: 0.19650450348854065
	round 22: 0.19786588847637177
	round 23: 0.1941422075033188
	round 24: 0.19867227971553802
	round 25: 0.19890561699867249
History (metrics, distributed, evaluate):
{'mse': [(1, 0.5030372142791748), (2, 0.45781099796295166), (3, 0.42085441946

In [16]:
splitting = [0.6, 0.1, 0.1, 0.1, 0.1]
do_fl(splitting)

Fold 1:
x_test: cpu
y_test: cpu
Client 0 trainset shape torch.Size([271450, 120])
Client 1 trainset shape torch.Size([45114, 120])
Client 2 trainset shape torch.Size([45226, 120])
Client 3 trainset shape torch.Size([45260, 120])
Client 4 trainset shape torch.Size([45246, 120])
FL started training on cpu
Number of available GPUs: 0


INFO flwr 2023-08-31 20:29:02,246 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)
2023-08-31 20:29:11,319	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
INFO flwr 2023-08-31 20:29:16,977 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 2302284596.0, 'object_store_memory': 1151142297.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-08-31 20:29:16,979 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 20:29:16,981 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 20:29:21,108 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 20:29:21,109 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 20:29:21,109 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 20:29:21,110 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


DEBUG flwr 2023-08-31 20:30:02,737 | server.py:187 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2023-08-31 20:30:02,738 | server.py:222 | fit_round 2: strategy sampled 2 clients (out of 5)


agrregate_evaluate results list length is  2
server round is  1
loss is  0.20976552367210388


...

agrregate_evaluate results list length is  2
server round is  24
loss is  0.19526755809783936


DEBUG flwr 2023-08-31 20:41:11,859 | server.py:236 | fit_round 25 received 2 results and 0 failures
DEBUG flwr 2023-08-31 20:41:11,879 | server.py:173 | evaluate_round 25: strategy sampled 2 clients (out of 5)


agrregate_fit results list length is  2
Saving round 25 aggregated_parameters...


DEBUG flwr 2023-08-31 20:41:13,651 | server.py:187 | evaluate_round 25 received 2 results and 0 failures
INFO flwr 2023-08-31 20:41:13,652 | server.py:153 | FL finished in 712.541729300021
INFO flwr 2023-08-31 20:41:13,662 | app.py:225 | app_fit: losses_distributed [(1, 0.20976552367210388), (2, 0.20281700789928436), (3, 0.20005740225315094), (4, 0.19868142902851105), (5, 0.1992044299840927), (6, 0.20088426768779755), (7, 0.2005656361579895), (8, 0.20206958055496216), (9, 0.2004166543483734), (10, 0.1989145278930664), (11, 0.20119903981685638), (12, 0.1968914121389389), (13, 0.2001321017742157), (14, 0.19921350479125977), (15, 0.1945384442806244), (16, 0.19557063281536102), (17, 0.19895200431346893), (18, 0.19821661710739136), (19, 0.19526979327201843), (20, 0.19850149750709534), (21, 0.1992805153131485), (22, 0.19497613608837128), (23, 0.19635678827762604), (24, 0.19526755809783936), (25, 0.19338931143283844)]
INFO flwr 2023-08-31 20:41:13,663 | app.py:226 | app_fit: metrics_distribut

agrregate_evaluate results list length is  2
server round is  25
loss is  0.19338931143283844
History (loss, distributed):
	round 1: 0.20976552367210388
	round 2: 0.20281700789928436
	round 3: 0.20005740225315094
	round 4: 0.19868142902851105
	round 5: 0.1992044299840927
	round 6: 0.20088426768779755
	round 7: 0.2005656361579895
	round 8: 0.20206958055496216
	round 9: 0.2004166543483734
	round 10: 0.1989145278930664
	round 11: 0.20119903981685638
	round 12: 0.1968914121389389
	round 13: 0.2001321017742157
	round 14: 0.19921350479125977
	round 15: 0.1945384442806244
	round 16: 0.19557063281536102
	round 17: 0.19895200431346893
	round 18: 0.19821661710739136
	round 19: 0.19526979327201843
	round 20: 0.19850149750709534
	round 21: 0.1992805153131485
	round 22: 0.19497613608837128
	round 23: 0.19635678827762604
	round 24: 0.19526755809783936
	round 25: 0.19338931143283844
History (metrics, distributed, evaluate):
{'mse': [(1, 0.41953104734420776), (2, 0.4056340157985687), (3, 0.40011480450

In [17]:
splitting = [0.8, 0.05, 0.05, 0.05, 0.05]
do_fl(splitting)

Fold 1:
x_test: cpu
y_test: cpu


INFO flwr 2023-08-31 20:41:15,373 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)


Client 0 trainset shape torch.Size([361840, 120])
Client 1 trainset shape torch.Size([18124, 120])
Client 2 trainset shape torch.Size([18132, 120])
Client 3 trainset shape torch.Size([18090, 120])
Client 4 trainset shape torch.Size([18044, 120])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 20:41:24,542	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
INFO flwr 2023-08-31 20:41:29,377 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 2187573659.0, 'object_store_memory': 1093786828.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-08-31 20:41:29,379 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 20:41:29,379 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 20:41:33,275 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 20:41:33,276 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 20:41:33,278 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 20:41:33,279 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 20:41:40,038 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 20:41:40,044 | fedavg

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  2
server round is  24
loss is  0.2000502049922943


DEBUG flwr 2023-08-31 20:51:21,916 | server.py:236 | fit_round 25 received 2 results and 0 failures
DEBUG flwr 2023-08-31 20:51:21,926 | server.py:173 | evaluate_round 25: strategy sampled 2 clients (out of 5)


agrregate_fit results list length is  2
Saving round 25 aggregated_parameters...


DEBUG flwr 2023-08-31 20:51:23,408 | server.py:187 | evaluate_round 25 received 2 results and 0 failures
INFO flwr 2023-08-31 20:51:23,410 | server.py:153 | FL finished in 590.1315968999988
INFO flwr 2023-08-31 20:51:23,411 | app.py:225 | app_fit: losses_distributed [(1, 0.26766642928123474), (2, 0.26124218106269836), (3, 0.2556231915950775), (4, 0.24825914204120636), (5, 0.24034053087234497), (6, 0.20377640426158905), (7, 0.20263846218585968), (8, 0.2006395012140274), (9, 0.2009199559688568), (10, 0.19958831369876862), (11, 0.20278628170490265), (12, 0.19871513545513153), (13, 0.19599415361881256), (14, 0.19767457246780396), (15, 0.19689708948135376), (16, 0.19816115498542786), (17, 0.19603005051612854), (18, 0.1982855349779129), (19, 0.19718149304389954), (20, 0.19609124958515167), (21, 0.19882932305335999), (22, 0.19677536189556122), (23, 0.1948392540216446), (24, 0.2000502049922943), (25, 0.1980738639831543)]
INFO flwr 2023-08-31 20:51:23,414 | app.py:226 | app_fit: metrics_distrib

agrregate_evaluate results list length is  2
server round is  25
loss is  0.1980738639831543
History (loss, distributed):
	round 1: 0.26766642928123474
	round 2: 0.26124218106269836
	round 3: 0.2556231915950775
	round 4: 0.24825914204120636
	round 5: 0.24034053087234497
	round 6: 0.20377640426158905
	round 7: 0.20263846218585968
	round 8: 0.2006395012140274
	round 9: 0.2009199559688568
	round 10: 0.19958831369876862
	round 11: 0.20278628170490265
	round 12: 0.19871513545513153
	round 13: 0.19599415361881256
	round 14: 0.19767457246780396
	round 15: 0.19689708948135376
	round 16: 0.19816115498542786
	round 17: 0.19603005051612854
	round 18: 0.1982855349779129
	round 19: 0.19718149304389954
	round 20: 0.19609124958515167
	round 21: 0.19882932305335999
	round 22: 0.19677536189556122
	round 23: 0.1948392540216446
	round 24: 0.2000502049922943
	round 25: 0.1980738639831543
History (metrics, distributed, evaluate):
{'mse': [(1, 0.5353328585624695), (2, 0.5224843621253967), (3, 0.511246383190

In [18]:
splitting = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
do_fl(splitting)

Fold 1:
x_test: cpu
y_test: cpu
Client 0 trainset shape torch.Size([45160, 120])
Client 1 trainset shape torch.Size([45402, 120])
Client 2 trainset shape torch.Size([45254, 120])
Client 3 trainset shape torch.Size([45252, 120])
Client 4 trainset shape torch.Size([45214, 120])
Client 5 trainset shape torch.Size([45086, 120])
Client 6 trainset shape torch.Size([45288, 120])


INFO flwr 2023-08-31 20:51:24,518 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)


Client 7 trainset shape torch.Size([45222, 120])
Client 8 trainset shape torch.Size([45302, 120])
Client 9 trainset shape torch.Size([45116, 120])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 20:51:33,546	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
INFO flwr 2023-08-31 20:51:38,425 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 2619902363.0, 'object_store_memory': 1309951180.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-08-31 20:51:38,427 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 20:51:38,429 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 20:51:42,433 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 20:51:42,434 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 20:51:42,435 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 20:51:42,436 | server.py:222 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 20:52:06,054 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-08-31 20:52:06,066 | fedav

agrregate_fit results list length is  5
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  5
server round is  24
loss is  0.1993088722229004


DEBUG flwr 2023-08-31 21:02:13,407 | server.py:236 | fit_round 25 received 5 results and 0 failures
DEBUG flwr 2023-08-31 21:02:13,433 | server.py:173 | evaluate_round 25: strategy sampled 5 clients (out of 10)


agrregate_fit results list length is  5
Saving round 25 aggregated_parameters...


DEBUG flwr 2023-08-31 21:02:18,512 | server.py:187 | evaluate_round 25 received 5 results and 0 failures
INFO flwr 2023-08-31 21:02:18,514 | server.py:153 | FL finished in 636.0787664999953
INFO flwr 2023-08-31 21:02:18,515 | app.py:225 | app_fit: losses_distributed [(1, 0.24608145654201508), (2, 0.24272729456424713), (3, 0.23020319640636444), (4, 0.21925793588161469), (5, 0.2124049812555313), (6, 0.20910555124282837), (7, 0.20631642639636993), (8, 0.20396791398525238), (9, 0.20480281114578247), (10, 0.20337146520614624), (11, 0.20283839106559753), (12, 0.20183557271957397), (13, 0.2011600285768509), (14, 0.20228014886379242), (15, 0.2008834332227707), (16, 0.2013113796710968), (17, 0.20090308785438538), (18, 0.20129132270812988), (19, 0.20034687221050262), (20, 0.199854776263237), (21, 0.20022404193878174), (22, 0.20032906532287598), (23, 0.19949375092983246), (24, 0.1993088722229004), (25, 0.19864104688167572)]
INFO flwr 2023-08-31 21:02:18,516 | app.py:226 | app_fit: metrics_distrib

agrregate_evaluate results list length is  5
server round is  25
loss is  0.19864104688167572
History (loss, distributed):
	round 1: 0.24608145654201508
	round 2: 0.24272729456424713
	round 3: 0.23020319640636444
	round 4: 0.21925793588161469
	round 5: 0.2124049812555313
	round 6: 0.20910555124282837
	round 7: 0.20631642639636993
	round 8: 0.20396791398525238
	round 9: 0.20480281114578247
	round 10: 0.20337146520614624
	round 11: 0.20283839106559753
	round 12: 0.20183557271957397
	round 13: 0.2011600285768509
	round 14: 0.20228014886379242
	round 15: 0.2008834332227707
	round 16: 0.2013113796710968
	round 17: 0.20090308785438538
	round 18: 0.20129132270812988
	round 19: 0.20034687221050262
	round 20: 0.199854776263237
	round 21: 0.20022404193878174
	round 22: 0.20032906532287598
	round 23: 0.19949375092983246
	round 24: 0.1993088722229004
	round 25: 0.19864104688167572
History (metrics, distributed, evaluate):
{'mse': [(1, 1.2304072827100754), (2, 1.2136364728212357), (3, 1.15101598203

In [19]:
splitting = [0.6, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04, 0.04]
do_fl(splitting)

Fold 1:
x_test: cpu
y_test: cpu


INFO flwr 2023-08-31 21:02:20,184 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)


Client 0 trainset shape torch.Size([271302, 120])
Client 1 trainset shape torch.Size([18080, 120])
Client 2 trainset shape torch.Size([18058, 120])
Client 3 trainset shape torch.Size([18058, 120])
Client 4 trainset shape torch.Size([18162, 120])
Client 5 trainset shape torch.Size([18130, 120])
Client 6 trainset shape torch.Size([18170, 120])
Client 7 trainset shape torch.Size([18052, 120])
Client 8 trainset shape torch.Size([18026, 120])
Client 9 trainset shape torch.Size([18110, 120])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 21:02:30,122	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
INFO flwr 2023-08-31 21:02:35,710 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 2931906971.0, 'object_store_memory': 1465953484.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-08-31 21:02:35,712 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 21:02:35,713 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 21:02:40,199 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 21:02:40,200 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 21:02:40,201 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 21:02:40,202 | server.py:222 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 21:02:53,113 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-08-31 21:02:53,128 | fedav

agrregate_fit results list length is  5
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  5
server round is  24
loss is  0.1955297440290451


DEBUG flwr 2023-08-31 21:16:03,579 | server.py:236 | fit_round 25 received 5 results and 0 failures
DEBUG flwr 2023-08-31 21:16:03,597 | server.py:173 | evaluate_round 25: strategy sampled 5 clients (out of 10)


agrregate_fit results list length is  5
Saving round 25 aggregated_parameters...


DEBUG flwr 2023-08-31 21:16:07,382 | server.py:187 | evaluate_round 25 received 5 results and 0 failures
INFO flwr 2023-08-31 21:16:07,384 | server.py:153 | FL finished in 807.1818660999998
INFO flwr 2023-08-31 21:16:07,386 | app.py:225 | app_fit: losses_distributed [(1, 0.27223438024520874), (2, 0.22088676691055298), (3, 0.20534662902355194), (4, 0.2049075812101364), (5, 0.20225346088409424), (6, 0.20058108866214752), (7, 0.19882579147815704), (8, 0.19986529648303986), (9, 0.19957146048545837), (10, 0.19781704246997833), (11, 0.20004689693450928), (12, 0.19951999187469482), (13, 0.19682270288467407), (14, 0.1983983963727951), (15, 0.19799776375293732), (16, 0.19781434535980225), (17, 0.19675888121128082), (18, 0.1977062225341797), (19, 0.19657933712005615), (20, 0.19749577343463898), (21, 0.1981920450925827), (22, 0.19772869348526), (23, 0.19865518808364868), (24, 0.1955297440290451), (25, 0.19481030106544495)]
INFO flwr 2023-08-31 21:16:07,387 | app.py:226 | app_fit: metrics_distribu

agrregate_evaluate results list length is  5
server round is  25
loss is  0.19481030106544495
History (loss, distributed):
	round 1: 0.27223438024520874
	round 2: 0.22088676691055298
	round 3: 0.20534662902355194
	round 4: 0.2049075812101364
	round 5: 0.20225346088409424
	round 6: 0.20058108866214752
	round 7: 0.19882579147815704
	round 8: 0.19986529648303986
	round 9: 0.19957146048545837
	round 10: 0.19781704246997833
	round 11: 0.20004689693450928
	round 12: 0.19951999187469482
	round 13: 0.19682270288467407
	round 14: 0.1983983963727951
	round 15: 0.19799776375293732
	round 16: 0.19781434535980225
	round 17: 0.19675888121128082
	round 18: 0.1977062225341797
	round 19: 0.19657933712005615
	round 20: 0.19749577343463898
	round 21: 0.1981920450925827
	round 22: 0.19772869348526
	round 23: 0.19865518808364868
	round 24: 0.1955297440290451
	round 25: 0.19481030106544495
History (metrics, distributed, evaluate):
{'mse': [(1, 1.3611719012260437), (2, 1.104433834552765), (3, 1.0267331451177

In [20]:
splitting = [0.8, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
do_fl(splitting)

Fold 1:
x_test: cpu
y_test: cpu


INFO flwr 2023-08-31 21:16:09,163 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=25, round_timeout=None)


Client 0 trainset shape torch.Size([361890, 120])
Client 1 trainset shape torch.Size([8972, 120])
Client 2 trainset shape torch.Size([9020, 120])
Client 3 trainset shape torch.Size([9010, 120])
Client 4 trainset shape torch.Size([9074, 120])
Client 5 trainset shape torch.Size([9090, 120])
Client 6 trainset shape torch.Size([9078, 120])
Client 7 trainset shape torch.Size([9022, 120])
Client 8 trainset shape torch.Size([9044, 120])
Client 9 trainset shape torch.Size([9062, 120])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 21:16:17,971	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8268 
INFO flwr 2023-08-31 21:16:22,968 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 3027092276.0, 'object_store_memory': 1513546137.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-08-31 21:16:22,969 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 21:16:22,970 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 21:16:27,612 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 21:16:27,613 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 21:16:27,614 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 21:16:27,615 | server.py:222 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 21:17:19,387 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-08-31 21:17:19,402 | fedav

agrregate_fit results list length is  5
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  5
server round is  24
loss is  0.19690564274787903


DEBUG flwr 2023-08-31 21:37:15,514 | server.py:236 | fit_round 25 received 5 results and 0 failures
DEBUG flwr 2023-08-31 21:37:15,530 | server.py:173 | evaluate_round 25: strategy sampled 5 clients (out of 10)


agrregate_fit results list length is  5
Saving round 25 aggregated_parameters...


DEBUG flwr 2023-08-31 21:37:19,045 | server.py:187 | evaluate_round 25 received 5 results and 0 failures
INFO flwr 2023-08-31 21:37:19,047 | server.py:153 | FL finished in 1251.4323288000014
INFO flwr 2023-08-31 21:37:19,049 | app.py:225 | app_fit: losses_distributed [(1, 0.21046647429466248), (2, 0.20405693352222443), (3, 0.19925129413604736), (4, 0.20008715987205505), (5, 0.20019109547138214), (6, 0.20039187371730804), (7, 0.19767464697360992), (8, 0.196367084980011), (9, 0.2012336105108261), (10, 0.19961166381835938), (11, 0.19827638566493988), (12, 0.19694314897060394), (13, 0.1946459412574768), (14, 0.19735732674598694), (15, 0.19792796671390533), (16, 0.1970977485179901), (17, 0.19826658070087433), (18, 0.1944005787372589), (19, 0.19332990050315857), (20, 0.1965285688638687), (21, 0.1971183717250824), (22, 0.19417281448841095), (23, 0.19778898358345032), (24, 0.19690564274787903), (25, 0.1920037716627121)]
INFO flwr 2023-08-31 21:37:19,050 | app.py:226 | app_fit: metrics_distribu

agrregate_evaluate results list length is  5
server round is  25
loss is  0.1920037716627121
History (loss, distributed):
	round 1: 0.21046647429466248
	round 2: 0.20405693352222443
	round 3: 0.19925129413604736
	round 4: 0.20008715987205505
	round 5: 0.20019109547138214
	round 6: 0.20039187371730804
	round 7: 0.19767464697360992
	round 8: 0.196367084980011
	round 9: 0.2012336105108261
	round 10: 0.19961166381835938
	round 11: 0.19827638566493988
	round 12: 0.19694314897060394
	round 13: 0.1946459412574768
	round 14: 0.19735732674598694
	round 15: 0.19792796671390533
	round 16: 0.1970977485179901
	round 17: 0.19826658070087433
	round 18: 0.1944005787372589
	round 19: 0.19332990050315857
	round 20: 0.1965285688638687
	round 21: 0.1971183717250824
	round 22: 0.19417281448841095
	round 23: 0.19778898358345032
	round 24: 0.19690564274787903
	round 25: 0.1920037716627121
History (metrics, distributed, evaluate):
{'mse': [(1, 1.0523323714733124), (2, 1.0202846676111221), (3, 0.99625647068023